### Import Libraries

In [ ]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

### Import Dataset

In [ ]:
df = pd.read_csv("/kaggle/input/managerial-accounting/managerial_accounting.csv")
df.head()

 **1. Calculate Total Revenue and Total Expenses**

In [ ]:
revenue = df[df["Transaction Type"] == "Revenue"]
expense = df[df["Transaction Type"] == "Expense"]

total_revenue = revenue["Transaction Amount"].sum()
total_expense = expense["Transaction Amount"].sum()

print("Total Revenue =", "$" + str("{:,}".format(total_revenue)))
print("Total Expense =", "$" + str("{:,}".format(total_expense)))

**2: Calculate Profit Margin**

In [ ]:
profit = total_revenue - total_expense
profit_margin = (profit / total_revenue) * 100
print("Profit Margin = {:.2f}".format(profit_margin) + "%")

**3. Calculate Net income by Year**

In [ ]:
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"])
df["Year"] = df["Transaction Date"].dt.year

year_income = pd.pivot_table(df, index = "Year", columns = "Transaction Type", values = "Transaction Amount")
year_income["Net Income"] = year_income["Revenue"] - year_income["Expense"]
year_income.columns.name = None

year_income["Net Income"] = year_income["Net Income"].apply(lambda x: f"${x:,.2f}")
year_income = year_income[["Net Income"]]
year_income

**4. Analyze Expenses by Category**

In [ ]:
expenses = df[df["Transaction Type"] == "Expense"]
category_expenses = expenses.groupby("Category")["Transaction Amount"].agg("sum").to_frame().sort_values(by = "Transaction Amount", ascending = False).reset_index()
category_expenses["Transaction Amount"] = category_expenses["Transaction Amount"].apply(lambda x: f"${x:,.2f}")
category_expenses.columns = ["Category", "Expense"]
category_expenses

**5. Identify The Project That Generated The Most Revenue**

In [ ]:
revenue = df[df["Transaction Type"] == "Revenue"]
project_revenue = revenue.groupby("Project")["Transaction Amount"].agg("sum").to_frame().sort_values(by = "Transaction Amount", ascending = False).reset_index()
project_revenue["Transaction Amount"] = project_revenue["Transaction Amount"].apply(lambda x: f"${x:,.2f}")
project_revenue

In [ ]:
print(project_revenue["Project"].iloc[0], "has generated most revenue", str(project_revenue["Transaction Amount"].iloc[0]))

**6. Which Department Had The Highest Expenses On Average Per Transaction**

In [ ]:
expense = df[df["Transaction Type"] == "Expense"]
department_expense = expense.groupby("Department")["Transaction Amount"].agg("mean").to_frame().sort_values(by = "Transaction Amount", ascending = False).reset_index()
department_expense.columns = ["Department", "Average Expense Per Transaction"]
department_expense

In [ ]:
print(department_expense["Department"].iloc[0], "Department Had The Highest Expense On Average Per Transaction =", "$" + str(department_expense["Average Expense Per Transaction"].iloc[0].round(2)))

**7. Find The Location That Contributed The Most To The Company's Revenue**

In [ ]:
location_revenue = revenue.groupby("Location")["Transaction Amount"].agg("sum").to_frame().sort_values(by = "Transaction Amount", ascending = False).reset_index()
location_revenue["Transaction Amount"] = location_revenue["Transaction Amount"].apply(lambda x: f"${x:,.2f}")
location_revenue.columns = ["Location", "Revenue"]
location_revenue

In [ ]:
print(location_revenue["Location"].iloc[0], "Contributed Most To Company's Revenue =", location_revenue["Revenue"].iloc[0])

**8. Return On Investment Over Years**

In [ ]:
roi = pd.pivot_table(df, index = "Year", columns = "Transaction Type", values = "Transaction Amount")
roi["Net Income"] = roi["Revenue"] - roi["Expense"]
roi["Total Assets"] = roi["Asset Purchase"] - roi["Liability Payment"]
roi["ROI"] = (roi["Net Income"] / roi["Total Assets"]) * 100
roi["ROI"] = roi["ROI"].apply(lambda x: f"{x:.2f}%")
roi

**9. Identify Seasonality In Revenue & Expenses**

In [ ]:
df["Month"] = df["Transaction Date"].dt.month

In [ ]:
revenue_expense = pd.pivot_table(df, index = ["Year", "Month"], columns = "Transaction Type", values = "Transaction Amount")
revenue_expense.columns.name = None
revenue_expense = revenue_expense.reset_index()
revenue_expense["Net Income"] = revenue_expense["Revenue"] - revenue_expense["Expense"]
revenue_expense["Date"] = pd.to_datetime(revenue_expense[['Year', 'Month']].assign(DAY=1))
revenue_expense.head(3)

In [ ]:
plt.figure(figsize = (20, 7))
sns.lineplot(x = "Date", y = "Revenue", data = revenue_expense, color = "green", linewidth = 3, label = "Revenue")
sns.lineplot(x = "Date", y = "Expense", data = revenue_expense, color = "red", linewidth = 3, label = "Expense")

plt.title("Seasonality in Revenue & Expenses Throughout time", size = 15, weight = "bold")
plt.xlabel("Date", size = 15, weight = "bold")
plt.ylabel("Amount", size = 15, weight = "bold")
plt.xticks(size = 12)
plt.yticks(size = 12)
plt.show()

**10. Which category had the highest average transaction amount?**

In [ ]:
category = df.groupby("Category")["Transaction Amount"].agg("sum").to_frame().sort_values(by = "Transaction Amount", ascending = False).reset_index()
category["Transaction Amount"] = category["Transaction Amount"].apply(lambda x: f"${x:,.2f}")
category